# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [6]:
import requests
import json

url = 'https://en.wikipedia.org/wiki/Data_science'

In [12]:
html=requests.get(url).content
html[0:600]

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>Data science - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XincrgpAIC4AAKZby-cAAACI","wgCSPNonce":!1,"wgC'

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [14]:
from bs4 import BeautifulSoup as bs

In [15]:
soup=bs(html,'lxml')
soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Data science - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XincrgpAIC4AAKZby-cAAACI","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Data_science","wgTitle":"Data science","wgCurRevisionId":937217331,"wgRevisionId":937217331,"wgArticleId":35458904,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","Use dmy dates from December 2012","All articles wi

In [42]:
linkstotales=[print(e.get('href')) for e in soup.find_all('a')]

None
#mw-head
#p-search
/wiki/Information_science
/wiki/Machine_learning
/wiki/Data_mining
/wiki/File:Kernel_Machine.svg
/wiki/Statistical_classification
/wiki/Cluster_analysis
/wiki/Regression_analysis
/wiki/Anomaly_detection
/wiki/Automated_machine_learning
/wiki/Association_rule_learning
/wiki/Reinforcement_learning
/wiki/Structured_prediction
/wiki/Feature_engineering
/wiki/Feature_learning
/wiki/Online_machine_learning
/wiki/Semi-supervised_learning
/wiki/Unsupervised_learning
/wiki/Learning_to_rank
/wiki/Grammar_induction
/wiki/Supervised_learning
/wiki/Statistical_classification
/wiki/Regression_analysis
/wiki/Decision_tree_learning
/wiki/Ensemble_learning
/wiki/Bootstrap_aggregating
/wiki/Boosting_(machine_learning)
/wiki/Random_forest
/wiki/K-nearest_neighbors_algorithm
/wiki/Linear_regression
/wiki/Naive_Bayes_classifier
/wiki/Artificial_neural_network
/wiki/Logistic_regression
/wiki/Perceptron
/wiki/Relevance_vector_machine
/wiki/Support-vector_machine
/wiki/Cluster_analysis

In [44]:
linkshttps=[print(e['href']) for e in soup.select('a[href^="http"]')]

https://arxiv.org/list/cs.LG/recent
https://en.wikipedia.org/w/index.php?title=Template:Machine_learning_bar&action=edit
http://euads.org
http://www.gfkl.org/welcome/
http://cacm.acm.org/magazines/2013/12/169933-data-science-and-prediction/fulltext
https://doi.org/10.1145%2F2500499
https://web.archive.org/web/20141109113411/http://cacm.acm.org/magazines/2013/12/169933-data-science-and-prediction/fulltext
http://simplystatistics.org/2013/12/12/the-key-word-in-data-science-is-not-data-it-is-science/
https://web.archive.org/web/20140102194117/http://simplystatistics.org/2013/12/12/the-key-word-in-data-science-is-not-data-it-is-science/
https://www.springer.com/book/9784431702085
https://doi.org/10.1007%2F978-4-431-65950-1_3
https://books.google.com/books?id=oGs_AQAAIAAJ
https://web.archive.org/web/20170320193019/https://books.google.com/books?id=oGs_AQAAIAAJ
https://doi.org/10.1126%2Fscience.1170411
https://magazine.amstat.org/blog/2015/10/01/asa-statement-on-the-role-of-statistics-in-dat

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [60]:
domain = 'http://wikipedia.org'
html=requests.get(domain).content
soup=bs(html,'lxml')
#soup

In [70]:
rel_links=[e.get('href') for e in soup.find_all('a') if (e.get('href')).startswith('//')]
rel_links[0:10]

['//en.wikipedia.org/',
 '//ja.wikipedia.org/',
 '//de.wikipedia.org/',
 '//es.wikipedia.org/',
 '//ru.wikipedia.org/',
 '//fr.wikipedia.org/',
 '//it.wikipedia.org/',
 '//zh.wikipedia.org/',
 '//pt.wikipedia.org/',
 '//pl.wikipedia.org/']

In [74]:
abs_links=[e['href'] for e in soup.select('a[href^="http"]')]
abs_links

['https://en.wikipedia.org/wiki/List_of_Wikipedia_mobile_applications',
 'https://creativecommons.org/licenses/by-sa/3.0/']

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [75]:
import os

In [83]:
folder= 'wikipedia'
#os.mkdir(folder) #crear carpeta
#os.chdir('C:\\Users\\Sativa\\test\\ironhack\\datamex0120\\module-1\\lab-parallelization\\your-code\\wikipedia') #cambiando de directorio de trabajo
os.getcwd() #comprobando ruta

'C:\\Users\\Sativa\\test\\ironhack\\datamex0120\\module-1\\lab-parallelization\\your-code\\wikipedia'

## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [89]:
#!pip3 install python-slugify 

In [90]:
from slugify import slugify

In [109]:
def index_page(link):
    try:
        html=request.get(link).content
        #print(html)
        slugified=slugify(link)
        newfile=open('slugified','w')
        newfile.write(html)
        newfile.close()
    except:
        pass

#este tema me resultó confuso

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [112]:
#%%time


UsageError: %%time is a cell magic, but the cell body is empty. Did you mean the line magic %time (single %)?


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [ ]:
import multiprocessing